# Global Dataset Overview

This notebook provides a comprehensive overview of the OpenPowerlifting dataset including counts, distributions, missing data analysis, and comparison of existing scoring systems.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from utils import categorize_ipf_weightclass, map_division_to_age_group, calculate_retirement_status, create_quality_filter, categorize_lifters, categorize_federation_testing_status

# Load the processed dataset
df = pd.read_parquet("../data/processed/full_dataset.parquet")
print(f"Dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")


Dataset loaded: 2,545,338 rows, 73 columns


## 0. Data Cleaning and Preparation


In [ ]:
# Convert Lbs to Kg and fill missing Kg values
# Conversion factor: 1 lb = 0.453592 kg
LBS_TO_KG = 0.453592

# Find all Lbs columns and their corresponding Kg columns
lbs_cols = [col for col in df.columns if col.endswith('Lbs') or 'Lbs' in col]
print(f"Found {len(lbs_cols)} Lbs columns to convert...")

for lbs_col in lbs_cols:
    # Find corresponding Kg column (e.g., TotalLbs -> TotalKg)
    kg_col = lbs_col.replace('Lbs', 'Kg')
    
    if kg_col in df.columns:
        # Convert Lbs to Kg where Kg is missing
        # Only convert where Lbs exists and Kg is missing
        mask = df[lbs_col].notna() & df[kg_col].isna()
        if mask.sum() > 0:
            df.loc[mask, kg_col] = df.loc[mask, lbs_col] * LBS_TO_KG
            print(f"  Converted {mask.sum():,} values from {lbs_col} to {kg_col}")
    else:
        # If no Kg column exists, create one from Lbs
        if df[lbs_col].notna().sum() > 0:
            df[kg_col] = df[lbs_col] * LBS_TO_KG
            print(f"  Created {kg_col} from {lbs_col} ({df[lbs_col].notna().sum():,} values)")

# Now remove Lbs columns after conversion
print(f"\nRemoving {len(lbs_cols)} Lbs columns (converted to Kg)...")
df = df.drop(columns=lbs_cols)
print(f"After removing Lbs columns: {len(df.columns)} columns")

# Add Age Group categorization from Division (needed for FailedMeet logic)
# This must be done BEFORE FailedMeet flagging
if 'Division' in df.columns:
    print("\nAdding Age Group categorization from Division (for FailedMeet logic)...")
    df['AgeGroup'] = df['Division'].apply(map_division_to_age_group)
    print("✓ Age Group categorization complete")
else:
    print("⚠ Warning: Cannot add Age Group - missing Division column")
    df['AgeGroup'] = None

# Flag failed meets based on multiple criteria:
# 1. TotalKg is missing/empty
# 2. TotalKg < 150kg AND AgeGroup is NOT 'Youth' (Youth can have legitimately low totals)
# 3. Bombed out: All attempts missing/zero in any of the three lift groups (squat, bench, or deadlift)
if 'TotalKg' in df.columns:
    # Check for bombed out lifters (all zeros/missing in any lift group)
    squat_cols = ['Squat1Kg', 'Squat2Kg', 'Squat3Kg']
    bench_cols = ['Bench1Kg', 'Bench2Kg', 'Bench3Kg']
    deadlift_cols = ['Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg']
    
    # Check if all squat attempts are missing/zero
    all_squat_missing = False
    if all(col in df.columns for col in squat_cols):
        all_squat_missing = (
            df[squat_cols].isna().all(axis=1) | 
            (df[squat_cols].fillna(0) == 0).all(axis=1)
        )
    
    # Check if all bench attempts are missing/zero
    all_bench_missing = False
    if all(col in df.columns for col in bench_cols):
        all_bench_missing = (
            df[bench_cols].isna().all(axis=1) | 
            (df[bench_cols].fillna(0) == 0).all(axis=1)
        )
    
    # Check if all deadlift attempts are missing/zero
    all_deadlift_missing = False
    if all(col in df.columns for col in deadlift_cols):
        all_deadlift_missing = (
            df[deadlift_cols].isna().all(axis=1) | 
            (df[deadlift_cols].fillna(0) == 0).all(axis=1)
        )
    
    # Combine bombed out conditions (bombed out if ALL attempts missing in ANY group)
    bombed_out = all_squat_missing | all_bench_missing | all_deadlift_missing
    
    df['FailedMeet'] = (
        df['TotalKg'].isna() |  # Missing total
        (
            (df['TotalKg'] < 150) &  # Low total
            (df['TotalKg'].notna()) &  # But total exists
            (df['AgeGroup'] != 'Youth')  # And not a Youth lifter
        ) |
        bombed_out  # Bombed out (all attempts missing in any lift group)
    )
    failed_count = df['FailedMeet'].sum()
    bombed_out_count = bombed_out.sum() if isinstance(bombed_out, pd.Series) else 0
    print(f"\nFlagged {failed_count:,} failed meets")
    print(f"  This represents {failed_count/len(df)*100:.2f}% of all entries")
    print(f"  Bombed out (all attempts missing in any lift group): {bombed_out_count:,}")
    print(f"  Criteria: Missing TotalKg OR (TotalKg < 150kg AND AgeGroup != 'Youth') OR bombed out")

# Ensure Sex column exists and filter to M/F only for gender analysis
if 'Sex' in df.columns:
    print(f"\nSex distribution:")
    print(df['Sex'].value_counts())
    # Create analysis dataframe (can include all, but we'll separate by M/F in analysis)
    df_analysis = df.copy()
else:
    print("⚠ Warning: Sex column not found")
    df_analysis = df.copy()

# Add IPF Weight Class categorization
if 'BodyweightKg' in df_analysis.columns and 'Sex' in df_analysis.columns:
    print("\nAdding IPF Weight Class categorization...")
    df_analysis['IPF_WeightClass'] = df_analysis.apply(
        lambda row: categorize_ipf_weightclass(row['BodyweightKg'], row['Sex']), 
        axis=1
    )
    
    # Show distribution of IPF weight classes
    print("\nIPF Weight Class distribution:")
    ipf_counts = df_analysis['IPF_WeightClass'].value_counts().sort_index()
    print(ipf_counts)
    
    # Show by gender
    print("\nIPF Weight Class by Gender:")
    if 'Sex' in df_analysis.columns:
        ipf_gender = pd.crosstab(df_analysis['Sex'], df_analysis['IPF_WeightClass'], margins=True)
        print(ipf_gender)
else:
    print("⚠ Warning: Cannot add IPF Weight Class - missing BodyweightKg or Sex column")
    df_analysis['IPF_WeightClass'] = None

# Age Group already calculated earlier (for FailedMeet logic)
# Just show the distribution
if 'AgeGroup' in df_analysis.columns:
    print("\nAge Group distribution (from Division - 100% coverage):")
    age_group_counts = df_analysis['AgeGroup'].value_counts()
    print(age_group_counts)
    
    # Show by gender
    if 'Sex' in df_analysis.columns:
        print("\nAge Group by Gender:")
        age_by_sex = pd.crosstab(df_analysis['Sex'], df_analysis['AgeGroup'], margins=True)
        print(age_by_sex)
    
    print("\n✓ Age Group categorization complete (using Division - 100% coverage)")
else:
    print("⚠ Warning: Age Group not found - should have been calculated earlier")
    if 'Division' in df_analysis.columns:
        df_analysis['AgeGroup'] = df_analysis['Division'].apply(map_division_to_age_group)
        print("  Calculated AgeGroup from Division as fallback")
    else:
        df_analysis['AgeGroup'] = None

# Calculate retirement status
print("\n" + "="*80)
print("CALCULATING RETIREMENT STATUS")
print("="*80)
df_analysis = calculate_retirement_status(df_analysis)
retired_count = df_analysis['Retired'].sum()
print(f"Flagged {retired_count:,} entries from retired lifters (4+ years since last meet)")
print(f"  This represents {retired_count/len(df_analysis)*100:.2f}% of all entries")

# Create quality filter (excludes outliers and invalid data)
print("\n" + "="*80)
print("CREATING QUALITY FILTER")
print("="*80)
quality_mask = create_quality_filter(df_analysis)
filtered_out = (~quality_mask).sum()
print(f"Quality filter will exclude {filtered_out:,} entries ({filtered_out/len(df_analysis)*100:.2f}%)")
print(f"  Remaining entries for analysis: {quality_mask.sum():,} ({quality_mask.sum()/len(df_analysis)*100:.2f}%)")

# Create clean dataset (filtered)
df_clean = df_analysis[quality_mask].copy()
print(f"\n✓ Using filtered dataset with {len(df_clean):,} entries")

# Categorize lifters (New/Intermediate/Advanced)
print("\n" + "="*80)
print("CATEGORIZING LIFTERS")
print("="*80)
df_clean = categorize_lifters(df_clean)
print("✓ Lifters categorized into New, Intermediate, and Advanced")

# Categorize federation testing status (Drug Tested vs Untested)
print("\n" + "="*80)
print("CATEGORIZING FEDERATION TESTING STATUS")
print("="*80)
df_clean = categorize_federation_testing_status(df_clean)
df_analysis = categorize_federation_testing_status(df_analysis)
testing_status_counts = df_clean['FederationTestingStatus'].value_counts()
print("Federation Testing Status distribution:")
print(testing_status_counts)
print(f"✓ Federation testing status categorized")

print(f"\n✓ Data cleaning complete")
print(f"Final dataset: {len(df_analysis):,} rows, {len(df_analysis.columns)} columns")
print(f"Clean dataset (filtered): {len(df_clean):,} rows")


Found 18 Lbs columns to convert...
  Converted 14,681 values from BodyweightLbs to BodyweightKg
  Converted 8,166 values from Deadlift1Lbs to Deadlift1Kg
  Converted 7,867 values from Deadlift2Lbs to Deadlift2Kg
  Converted 7,150 values from Deadlift3Lbs to Deadlift3Kg
  Converted 531 values from Deadlift4Lbs to Deadlift4Kg
  Converted 11,896 values from Best3DeadliftLbs to Best3DeadliftKg
  Converted 14,726 values from TotalLbs to TotalKg
  Converted 5,484 values from Squat1Lbs to Squat1Kg
  Converted 5,424 values from Squat2Lbs to Squat2Kg
  Converted 5,243 values from Squat3Lbs to Squat3Kg
  Converted 8,214 values from Best3SquatLbs to Best3SquatKg
  Converted 8,572 values from Bench1Lbs to Bench1Kg
  Converted 8,164 values from Bench2Lbs to Bench2Kg
  Converted 7,403 values from Bench3Lbs to Bench3Kg
  Converted 12,702 values from Best3BenchLbs to Best3BenchKg
  Converted 376 values from Bench4Lbs to Bench4Kg
  Converted 40 values from Squat4Lbs to Squat4Kg

Removing 18 Lbs columns

## 1. Basic Counts


In [3]:
# Count unique lifters, meets, federations
unique_lifters = df['Name'].nunique() if 'Name' in df.columns else 0
unique_meets = df['MeetPath'].nunique() if 'MeetPath' in df.columns else 0
unique_federations = df['Federation'].nunique() if 'Federation' in df.columns else 0

print("=== Dataset Overview ===")
print(f"Total entries: {len(df):,}")
print(f"Unique lifters: {unique_lifters:,}")
print(f"Unique meets: {unique_meets:,}")
print(f"Unique federations: {unique_federations:,}")
print(f"Date range: {df['MeetDate'].min()} to {df['MeetDate'].max()}")


=== Dataset Overview ===
Total entries: 2,545,338
Unique lifters: 664,572
Unique meets: 38,880
Unique federations: 390
Date range: 2015-01-01 00:00:00 to 2025-12-27 00:00:00


## 2. Distribution Analysis

### 2.0. Overall Distribution Statistics (Gender × Weight Class × Age Group(teen-masters 1 only?) x Ruleset(raw vs equipment) x Federation(tested vs untested))


In [4]:
# Key distributions - SEPARATED BY GENDER, IPF WEIGHT CLASS, AGE GROUP, AND FEDERATION TESTING STATUS
# Overall distributions are removed as they skew data - all analysis must be gender × weight class × age group × testing status separated

key_cols = ['TotalKg', 'BodyweightKg', 'Wilks', 'Dots', 'Goodlift']

print("=== Distribution Statistics by Gender, IPF Weight Class, Age Group, and Federation Testing Status ===")
print("Note: Overall distributions removed - all analysis is separated by (Gender × Weight Class × Age Group × Testing Status)")
print("      to prevent data skewing from combining different weight classes, age groups, and testing statuses.\n")
print("      Distributions are split into: Drug Tested Federations vs Untested Federations\n")

# Check if we have the required columns
if 'IPF_WeightClass' not in df_analysis.columns or 'Sex' not in df_analysis.columns or 'AgeGroup' not in df_analysis.columns or 'FederationTestingStatus' not in df_analysis.columns:
    print("⚠ Warning: IPF_WeightClass, Sex, or AgeGroup column missing. Cannot perform separated analysis.")
    print("   Falling back to overall statistics...")
    for col in key_cols:
        if col in df.columns:
            print(f"\n{col} (Overall - NOT RECOMMENDED):")
            print(df[col].describe())
            print(f"  Missing: {df[col].isna().sum():,} ({df[col].isna().sum()/len(df)*100:.1f}%)")
else:
    # Filter to entries with IPF weight class, gender, age group, and testing status
    valid_df = df_analysis[
        (df_analysis['IPF_WeightClass'].notna()) & 
        (df_analysis['Sex'].isin(['M', 'F'])) &
        (df_analysis['AgeGroup'].notna()) &
        (df_analysis['FederationTestingStatus'].isin(['Drug Tested', 'Untested']))
    ].copy()
    
    print(f"Entries with valid IPF Weight Class, Gender, Age Group, and Testing Status: {len(valid_df):,} ({len(valid_df)/len(df_analysis)*100:.1f}%)\n")
    
    # Show split by testing status
    testing_split = valid_df['FederationTestingStatus'].value_counts()
    print("Entries by Federation Testing Status:")
    for status, count in testing_split.items():
        print(f"  {status}: {count:,} ({count/len(valid_df)*100:.1f}%)")
    print()
    
    # Get all unique (Gender × Weight Class × Age Group × Testing Status) combinations
    gender_wc_age_test_combos = valid_df.groupby(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'], observed=True).size().reset_index(name='count')
    gender_wc_age_test_combos = gender_wc_age_test_combos.sort_values(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'])
    
    print(f"Total (Gender × Weight Class × Age Group × Testing Status) combinations: {len(gender_wc_age_test_combos)}")
    print("\nTop 20 combinations with sufficient data (>100 entries):")
    sufficient = gender_wc_age_test_combos[gender_wc_age_test_combos['count'] >= 100].head(20)
    print(sufficient.to_string(index=False))
    
    # For each key column, show summary by (Gender × Weight Class × Age Group × Testing Status)
    print("\n" + "="*80)
    print("DETAILED STATISTICS BY GENDER, WEIGHT CLASS, AGE GROUP, AND TESTING STATUS")
    print("="*80)
    print("Note: Statistics are shown separately for Drug Tested and Untested federations")
    print("="*80)
    
    # Standard age group order
    age_group_order = ['Youth', 'Teen', 'Sub-Junior', 'Junior', 'Open', 'Masters I', 'Masters II', 'Masters III', 'Masters IV']
    
    for col in key_cols:
        if col in valid_df.columns:
            print(f"\n{'='*80}")
            print(f"{col.upper()} - Statistics by (Gender × IPF Weight Class × Age Group × Testing Status)")
            print(f"{'='*80}")
            
            # Group by Sex, IPF_WeightClass, AgeGroup, and FederationTestingStatus
            stats = valid_df.groupby(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'], observed=True)[col].agg([
                'count', 'mean', 'median', 'std', 'min', 'max'
            ]).round(1)
            
            # Filter to groups with at least 10 entries for meaningful statistics
            counts = valid_df.groupby(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'], observed=True)[col].count()
            valid_groups = counts[counts >= 10].index
            
            if len(valid_groups) > 0:
                stats_filtered = stats.loc[valid_groups]
                
                # Show statistics organized by Gender and Weight Class, then Age Group
                men_classes = ['59kg', '66kg', '74kg', '83kg', '93kg', '105kg', '120kg', '120+kg']
                women_classes = ['47kg', '52kg', '57kg', '63kg', '69kg', '76kg', '84kg', '84+kg']
                
                # Show statistics split by Testing Status (Drug Tested vs Untested)
                for testing_status in ['Drug Tested', 'Untested']:
                    print(f"\n{'='*80}")
                    print(f"{testing_status.upper()} FEDERATIONS")
                    print(f"{'='*80}")
                    
                    for sex, classes, label in [('M', men_classes, 'MALE'), ('F', women_classes, 'FEMALE')]:
                        print(f"\n{label}:")
                        print("-" * 80)
                        
                        for wc in classes:
                            # Get stats for this weight class, gender, and testing status
                            try:
                                wc_stats = stats_filtered.xs((sex, wc, testing_status), level=['Sex', 'IPF_WeightClass', 'FederationTestingStatus'])
                                
                                if len(wc_stats) > 0:
                                    # Reorder by age group order
                                    available_ages = [ag for ag in age_group_order if ag in wc_stats.index]
                                    wc_stats = wc_stats.reindex(available_ages)
                                    
                                    print(f"\n  {label} - {wc}:")
                                    print(f"  {'Age Group':<15} {'Count':<8} {'Mean':<10} {'Median':<10} {'Std':<10} {'Min':<10} {'Max':<10}")
                                    print(f"  {'-'*15} {'-'*8} {'-'*10} {'-'*10} {'-'*10} {'-'*10} {'-'*10}")
                                    for ag, row in wc_stats.iterrows():
                                        print(f"  {str(ag):<15} {int(row['count']):<8} {row['mean']:<10.1f} {row['median']:<10.1f} {row['std']:<10.1f} {row['min']:<10.1f} {row['max']:<10.1f}")
                            except (KeyError, IndexError):
                                # No data for this weight class/gender/testing status combination
                                pass
                
                # Show missing data by group
                missing_by_group = valid_df.groupby(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'], observed=True)[col].apply(
                    lambda x: x.isna().sum()
                )
                missing_by_group = missing_by_group[missing_by_group > 0]
                if len(missing_by_group) > 0:
                    print(f"\nMissing data by group (showing top 20):")
                    print(missing_by_group.head(20).to_string())
            else:
                print(f"⚠ No groups with sufficient data (≥10 entries) for {col}")


=== Distribution Statistics by Gender, IPF Weight Class, Age Group, and Federation Testing Status ===
Note: Overall distributions removed - all analysis is separated by (Gender × Weight Class × Age Group × Testing Status)
      to prevent data skewing from combining different weight classes, age groups, and testing statuses.

      Distributions are split into: Drug Tested Federations vs Untested Federations

Entries with valid IPF Weight Class, Gender, Age Group, and Testing Status: 2,484,990 (97.6%)

Entries by Federation Testing Status:
  Untested: 2,273,612 (91.5%)
  Drug Tested: 211,378 (8.5%)

Total (Gender × Weight Class × Age Group × Testing Status) combinations: 224

Top 20 combinations with sufficient data (>100 entries):
Sex IPF_WeightClass    AgeGroup FederationTestingStatus  count
  F            47kg      Junior             Drug Tested    906
  F            47kg      Junior                Untested   8450
  F            47kg   Masters I                Untested   1888
  F   

### 2. Comprehensive Visualizations by All Dimensions (Gender × Weight Class × Age Group × Testing Status x Ruleset x Federation)


In [ ]:
# Comprehensive TotalKg Distribution by All Dimensions
# Breakdown: Gender × Weight Class × Age Group × Federation Testing Status

required_cols = ['TotalKg', 'Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus']
if all(col in df_clean.columns for col in required_cols):
    valid_df = df_clean[
        (df_clean['TotalKg'].notna()) &
        (df_clean['Sex'].isin(['M', 'F'])) &
        (df_clean['IPF_WeightClass'].notna()) &
        (df_clean['AgeGroup'].notna()) &
        (df_clean['FederationTestingStatus'].notna())
    ].copy()
    
    if len(valid_df) > 0:
        print("="*80)
        print("COMPREHENSIVE TOTALKG DISTRIBUTIONS BY ALL DIMENSIONS")
        print("="*80)
        print(f"Valid entries: {len(valid_df):,}")
        print(f"Breakdown: Gender × Weight Class × Age Group × Federation Testing Status\n")
        
        men_classes = ['59kg', '66kg', '74kg', '83kg', '93kg', '105kg', '120kg', '120+kg']
        women_classes = ['47kg', '52kg', '57kg', '63kg', '69kg', '76kg', '84kg', '84+kg']
        age_groups = ['Sub-Junior', 'Teen', 'Junior', 'Open', 'Masters I', 'Masters II', 'Masters III', 'Masters IV']
        testing_statuses = ['Drug Tested', 'Untested']
        
        # Generate separate visualizations for EACH (Gender × Weight Class × Age Group × Testing Status) combination
        print("Generating individual visualizations for each combination...")
        
        # Get all unique combinations
        combo_counts = valid_df.groupby(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'], observed=True).size()
        all_combos = combo_counts.index.tolist()
        
        print(f"Total combinations to visualize: {len(all_combos)}")
        
        # Create visualizations for each combination
        # Group combinations for efficient plotting
        plots_created = 0
        for combo in all_combos:
            sex, wc, age_group, test_status = combo
            combo_data = valid_df[
                (valid_df['Sex'] == sex) &
                (valid_df['IPF_WeightClass'] == wc) &
                (valid_df['AgeGroup'] == age_group) &
                (valid_df['FederationTestingStatus'] == test_status)
            ]['TotalKg'].dropna()
            
            if len(combo_data) > 0:
                # Create individual plot for this combination
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                ax.hist(combo_data, bins=30, edgecolor='black', alpha=0.7)
                ax.set_title(f'{sex} - {wc} - {age_group} - {test_status}\n(n={len(combo_data):,})', 
                           fontsize=12, fontweight='bold')
                ax.set_xlabel('TotalKg (kg)', fontsize=10)
                ax.set_ylabel('Frequency', fontsize=10)
                ax.grid(True, alpha=0.3)
                
                # Save individual plot
                safe_combo_name = f"{sex}_{wc}_{age_group}_{test_status}".replace(' ', '_').replace('+', 'plus')
                plt.tight_layout()
                plt.savefig(f'../data/processed/totalkg_{safe_combo_name}.png', dpi=150, bbox_inches='tight')
                plt.close()  # Close to free memory
                plots_created += 1
        
        print(f"✓ Created {plots_created} individual visualizations")
        print("  Files saved as: ../data/processed/totalkg_[combination].png")
    else:
        print("⚠ No valid entries with all required dimensions")
else:
    missing_cols = [col for col in required_cols if col not in df_clean.columns]
    print(f"⚠ Missing required columns: {missing_cols}")


COMPREHENSIVE TOTALKG DISTRIBUTIONS BY ALL DIMENSIONS
Valid entries: 701,028
Breakdown: Gender × Weight Class × Age Group × Federation Testing Status

Generating individual visualizations for each combination...
Total combinations to visualize: 223
✓ Created 223 individual visualizations
  Files saved as: ../data/processed/totalkg_[combination].png


In [ ]:
# Distribution Summary Statistics Table
# Comprehensive statistics broken down by all dimension combinations

required_cols = ['TotalKg', 'Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus']
if all(col in df_clean.columns for col in required_cols):
    valid_df = df_clean[
        (df_clean['TotalKg'].notna()) &
        (df_clean['Sex'].isin(['M', 'F'])) &
        (df_clean['IPF_WeightClass'].notna()) &
        (df_clean['AgeGroup'].notna()) &
        (df_clean['FederationTestingStatus'].notna())
    ].copy()
    
    if len(valid_df) > 0:
        print("="*80)
        print("DISTRIBUTION SUMMARY STATISTICS BY ALL DIMENSIONS")
        print("="*80)
        print(f"Valid entries: {len(valid_df):,}\n")
        
        # Calculate comprehensive statistics
        stats_df = valid_df.groupby(['Sex', 'IPF_WeightClass', 'AgeGroup', 'FederationTestingStatus'], observed=True)['TotalKg'].agg([
            'count', 'mean', 'median', 'std', 'min', 'max',
            lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)
        ]).round(1)
        
        stats_df.columns = ['Count', 'Mean', 'Median', 'Std', 'Min', 'Max', 'Q25', 'Q75']
        
        # Filter to combinations with sufficient data
        stats_df = stats_df[stats_df['Count'] >= 50]
        
        print(f"Total combinations with ≥50 entries: {len(stats_df)}\n")
        print("Full statistics table (all rows):")
        print("="*120)
        print(stats_df.to_string())
        
        # Save to CSV
        output_path = Path("../data/processed/distribution_summary_stats_by_all_dimensions.csv")
        stats_df.to_csv(output_path)
        print(f"\n✓ Saved full statistics table to: {output_path}")
        print(f"  Total rows: {len(stats_df)}")
        
        # Show summary by dimension
        print("\n" + "="*80)
        print("SUMMARY BY DIMENSION")
        print("="*80)
        
        # By Gender
        print("\nBy Gender:")
        print(valid_df.groupby('Sex', observed=True)['TotalKg'].agg(['count', 'mean', 'median']).round(1))
        
        # By Weight Class
        print("\nBy Weight Class:")
        wc_stats = valid_df.groupby('IPF_WeightClass', observed=True)['TotalKg'].agg(['count', 'mean', 'median']).round(1)
        print(wc_stats.sort_index())
        
        # By Age Group
        print("\nBy Age Group:")
        age_stats = valid_df.groupby('AgeGroup', observed=True)['TotalKg'].agg(['count', 'mean', 'median']).round(1)
        print(age_stats)
        
        # By Testing Status
        print("\nBy Federation Testing Status:")
        test_stats = valid_df.groupby('FederationTestingStatus', observed=True)['TotalKg'].agg(['count', 'mean', 'median']).round(1)
        print(test_stats)
        
    else:
        print("⚠ No valid entries with all required dimensions")
else:
    missing_cols = [col for col in required_cols if col not in df_clean.columns]
    print(f"⚠ Missing required columns: {missing_cols}")


DISTRIBUTION SUMMARY STATISTICS BY ALL DIMENSIONS
Valid entries: 701,028

Total combinations with ≥50 entries: 199

Full statistics table (all rows):
                                                         Count   Mean  Median    Std    Min     Max    Q25    Q75
Sex IPF_WeightClass AgeGroup    FederationTestingStatus                                                          
F   47kg            Junior      Drug Tested                192  270.6   270.0   55.2  155.0   417.5  232.5  310.0
                                Untested                  3072  282.5   280.5   53.3  152.5   505.0  245.0  317.5
                    Masters I   Untested                   879  272.9   272.5   52.9  150.0   455.0  235.0  307.5
                    Masters II  Drug Tested                 65  239.8   235.0   41.6  167.5   332.5  213.5  260.0
                                Untested                   690  257.9   255.2   48.9  150.0   447.5  222.5  285.0
                    Masters III Drug Tested         

## 3. Missing Data Analysis


In [ ]:
# Data Quality Flags - SEPARATED BY (Gender × IPF Weight Class)
# Overall quality flags removed - all analysis must be separated to prevent data skewing

if 'IPF_WeightClass' in df_analysis.columns and 'Sex' in df_analysis.columns:
    valid_df = df_analysis[df_analysis['Sex'].isin(['M', 'F'])].copy()
    
    print("=== Data Quality Flags - Separated by (Gender × IPF Weight Class) ===")
    print(f"Valid entries: {len(valid_df):,}\n")
    
    men_classes = ['59kg', '66kg', '74kg', '83kg', '93kg', '105kg', '120kg', '120+kg']
    women_classes = ['47kg', '52kg', '57kg', '63kg', '69kg', '76kg', '84kg', '84+kg']
    
    # TotalKg ranges: <100, 100-200, 200-300, 300-400, 400-500, 500-600, 600-700, 700-800, 800-900, 900-1000, 1000+kg
    total_ranges = [(0, 100), (100, 200), (200, 300), (300, 400), (400, 500), (500, 600), 
                   (600, 700), (700, 800), (800, 900), (900, 1000), (1000, 10000)]
    total_labels = ['<100', '100-200', '200-300', '300-400', '400-500', '500-600', 
                   '600-700', '700-800', '800-900', '900-1000', '1000+']
    
    # Age ranges: <14, 14-17, 18-22, 23-29, 30-39, 40-49, 50-59, 60-69, 70+
    age_ranges = [(0, 14), (14, 18), (18, 23), (23, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 150)]
    age_labels = ['<14', '14-17', '18-22', '23-29', '30-39', '40-49', '50-59', '60-69', '70+']
    
    print("TotalKg Ranges by (Gender × IPF Weight Class):")
    print("="*80)
    
    for sex, classes, label in [('M', men_classes, 'MALE'), ('F', women_classes, 'FEMALE')]:
        print(f"\n{label}:")
        print("-" * 80)
        
        for wc in classes:
            group_df = valid_df[(valid_df['Sex'] == sex) & (valid_df['IPF_WeightClass'] == wc) & 
                               (valid_df['TotalKg'].notna())]
            
            if len(group_df) >= 10:
                print(f"\n  {label} - {wc} (n={len(group_df):,}):")
                print(f"  {'TotalKg Range':<15} {'Count':<10} {'Percent':<10}")
                print(f"  {'-'*15} {'-'*10} {'-'*10}")
                
                for (low, high), range_label in zip(total_ranges, total_labels):
                    count = len(group_df[(group_df['TotalKg'] >= low) & (group_df['TotalKg'] < high)])
                    pct = (count / len(group_df) * 100) if len(group_df) > 0 else 0
                    if count > 0:
                        print(f"  {range_label:<15} {count:<10} {pct:<10.1f}%")
    
    print("\n\nAge Ranges by (Gender × IPF Weight Class):")
    print("="*80)
    
    for sex, classes, label in [('M', men_classes, 'MALE'), ('F', women_classes, 'FEMALE')]:
        print(f"\n{label}:")
        print("-" * 80)
        
        for wc in classes:
            group_df = valid_df[(valid_df['Sex'] == sex) & (valid_df['IPF_WeightClass'] == wc) & 
                               (valid_df['Age'].notna())]
            
            if len(group_df) >= 10:
                print(f"\n  {label} - {wc} (n={len(group_df):,}):")
                print(f"  {'Age Range':<12} {'Count':<10} {'Percent':<10}")
                print(f"  {'-'*12} {'-'*10} {'-'*10}")
                
                for (low, high), range_label in zip(age_ranges, age_labels):
                    count = len(group_df[(group_df['Age'] >= low) & (group_df['Age'] < high)])
                    pct = (count / len(group_df) * 100) if len(group_df) > 0 else 0
                    if count > 0:
                        print(f"  {range_label:<12} {count:<10} {pct:<10.1f}%")
    
    # Failed meets by group
    print("\n\nFailed Meets by (Gender × IPF Weight Class):")
    print("="*80)
    
    if 'FailedMeet' in valid_df.columns:
        for sex, classes, label in [('M', men_classes, 'MALE'), ('F', women_classes, 'FEMALE')]:
            print(f"\n{label}:")
            print("-" * 80)
            print(f"{'Weight Class':<15} {'Total':<10} {'Failed':<10} {'Failed%':<10}")
            print("-" * 80)
            
            for wc in classes:
                group_df = valid_df[(valid_df['Sex'] == sex) & (valid_df['IPF_WeightClass'] == wc)]
                
                if len(group_df) >= 10:
                    failed_count = group_df['FailedMeet'].sum() if 'FailedMeet' in group_df.columns else 0
                    failed_pct = (failed_count / len(group_df) * 100) if len(group_df) > 0 else 0
                    print(f"{wc:<15} {len(group_df):<10} {failed_count:<10} {failed_pct:<10.1f}%")
    
    print("\n✓ Data quality flags analysis complete")
else:
    print("⚠ Missing required columns (IPF_WeightClass or Sex) for separated analysis")


=== Data Quality Flags - Separated by (Gender × IPF Weight Class) ===
Valid entries: 2,545,188

TotalKg Ranges by (Gender × IPF Weight Class):

MALE:
--------------------------------------------------------------------------------

  MALE - 59kg (n=183,601):
  TotalKg Range   Count      Percent   
  --------------- ---------- ----------
  <100            17754      9.7       %
  100-200         25401      13.8      %
  200-300         28835      15.7      %
  300-400         54917      29.9      %
  400-500         40991      22.3      %
  500-600         13333      7.3       %
  600-700         2075       1.1       %
  700-800         282        0.2       %
  800-900         13         0.0       %

  MALE - 66kg (n=193,807):
  TotalKg Range   Count      Percent   
  --------------- ---------- ----------
  <100            7734       4.0       %
  100-200         33343      17.2      %
  200-300         12373      6.4       %
  300-400         31387      16.2      %
  400-500         57

In [ ]:
# Identify extreme values and potential data quality issues
print("=== Data Quality Checks ===")

# Extreme totals
if 'TotalKg' in df.columns:
    extreme_high = df[df['TotalKg'] > 2000]
    extreme_low = df[df['TotalKg'] < 50]
    print(f"Extremely high totals (>2000kg): {len(extreme_high)}")
    print(f"Extremely low totals (<50kg): {len(extreme_low)}")

# Extreme bodyweights
if 'BodyweightKg' in df.columns:
    extreme_bw_high = df[df['BodyweightKg'] > 200]
    extreme_bw_low = df[df['BodyweightKg'] < 30]
    print(f"Extremely high bodyweight (>200kg): {len(extreme_bw_high)}")
    print(f"Extremely low bodyweight (<30kg): {len(extreme_bw_low)}")

# Extreme ages
if 'Age' in df.columns:
    extreme_age_high = df[df['Age'] > 100]
    extreme_age_low = df[df['Age'] < 10]
    print(f"Extremely high age (>100): {len(extreme_age_high)}")
    print(f"Extremely low age (<10): {len(extreme_age_low)}")

# Duplicate entries (same name, same meet, same total)
if 'Name' in df.columns and 'MeetPath' in df.columns and 'TotalKg' in df.columns:
    duplicates = df.duplicated(subset=['Name', 'MeetPath', 'TotalKg'], keep=False)
    print(f"Potential duplicate entries: {duplicates.sum()}")


=== Data Quality Checks ===
Extremely high totals (>2000kg): 0
Extremely low totals (<50kg): 19361
Extremely high bodyweight (>200kg): 494
Extremely low bodyweight (<30kg): 967
Extremely high age (>100): 0
Extremely low age (<10): 888
Potential duplicate entries: 276502
